<a href="https://colab.research.google.com/github/Ciastkk45/kursAI-sb10/blob/main/7_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
df = sns.load_dataset('titanic')
print(f"Pierwsze 10 wpisów z zestawu  Titanic:")
print(df.head(10))

Pierwsze 10 wpisów z zestawu  Titanic:
   survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0         0       3    male  22.0      1      0   7.2500        S   Third   
1         1       1  female  38.0      1      0  71.2833        C   First   
2         1       3  female  26.0      0      0   7.9250        S   Third   
3         1       1  female  35.0      1      0  53.1000        S   First   
4         0       3    male  35.0      0      0   8.0500        S   Third   
5         0       3    male   NaN      0      0   8.4583        Q   Third   
6         0       1    male  54.0      0      0  51.8625        S   First   
7         0       3    male   2.0      3      1  21.0750        S   Third   
8         1       3  female  27.0      0      2  11.1333        S   Third   
9         1       2  female  14.0      1      0  30.0708        C  Second   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  wo

In [ ]:
print(f"Liczba brakujących wartości przed czyszczeniem:")
print(df.isnull().sum())

Liczba brakujących wartości przed czyszczeniem:
survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [ ]:
df = df.dropna(subset=["age"])
print(f"Liczba brakujących wartości po czyszczeniu (dla kolumny age):")
print(df.isnull().sum())

Liczba brakujących wartości po czyszczeniu (dla kolumny age):
survived         0
pclass           0
sex              0
age              0
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           530
embark_town      2
alive            0
alone            0
dtype: int64


In [8]:
print(f"Min bliet:{df['fare'].min()}")
print(f"Max bliet:{df['fare'].max()}")

def categorized_fare(fare, fare_range):
  if fare <= fare_range[0]:
    return 0 # kat niska biletu
  elif fare <= fare_range[1]:
    return 1 # kat średnia biletu
  else:
    return 2 # kat wysoka biletu
  pass
min_fare = df['fare'].min()
max_fare = df['fare'].max()
fare_step = (max_fare - min_fare) / 3
fare_range = [min_fare + fare_step, min_fare + 2 * fare_step]
print(f"Przedziały cenowe biletów:")
print(f"Niska:{min_fare:.2f} - {fare_range[0]:.2f}")
print(f"Średnia:{fare_range[0]:.2f} - {fare_range[1]:.2f}")
print(f"Wysoka:{fare_range[1]:.2f} - {max_fare:.2f}")

Min bliet:0.0
Max bliet:512.3292
Przedziały cenowe biletów:
Niska:0.00 - 170.78
Średnia:170.78 - 341.55
Wysoka:341.55 - 512.33


In [16]:
selected_features = ["sex", "age", "pclass", "fare_category"]
df_model = df.copy()
df_model["sex"] = df_model["sex"].map({"male": 0, "female": 1})
df_model["fare_category"] = df_model["fare"].apply(lambda x: categorized_fare(x, fare_range))
print(df_model.groupby(["fare_category"]).count())


               survived  pclass  sex  age  sibsp  parch  fare  embarked  \
fare_category                                                             
0                   696     696  696  696    696    696   696       694   
1                    15      15   15   15     15     15    15        15   
2                     3       3    3    3      3      3     3         3   

               class  who  adult_male  deck  embark_town  alive  alone  
fare_category                                                           
0                696  696         696   168          694    696    696  
1                 15   15          15    14           15     15     15  
2                  3    3           3     2            3      3      3  


In [21]:
x = df_model[selected_features]
y = df_model["survived"]
x_train,x_test, y_train,y_test = train_test_split(x, y, test_size=0.2, random_state = 42)


In [20]:
#standaryzajca danych
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)